In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import xlrd

## Load Data into Mongo

In [2]:
# read in spreadsheet
df = pd.read_excel('TestData.xlsx', skiprows=2, engine='openpyxl')
df.head(2)

,ACTIVITYID,PGID,OID,RPID,PATID,USERID,SESSIONID,WEBSERVERIP,REMOTEIP,PAGENAME,CREATEDON
0,1875082,40081,100,0,0,USER40081,pgdwm055edgvwqf5kb2rzg55,XXX.XX.XXX.110,172.130.400.165,Login - 172.130.400.165,2019-03-24 18:04:29.030
1,1875083,40081,100,0,0,USER40081,pgdwm055edgvwqf5kb2rzg55,XXX.XX.XXX.110,172.130.400.165,SearchPatients,2019-03-24 18:04:29.077


In [5]:
from pymongo import MongoClient
import pprint

In [6]:
# Connect to database
client = MongoClient('mongodb://localhost:27017/')

In [9]:
# Create a collection
db = client.testdata
collection = db['rawdata']

In [51]:
row.to_dict()

{'ACTIVITYID': 2074074,
 'PGID': 40081,
 'OID': 100,
 'RPID': 155,
 'PATID': 163,
 'USERID': 'USER_N',
 'SESSIONID': 'mrcjgjm2kvrakv45mcxelgbk',
 'WEBSERVERIP': 'XXX.XX.XXX.107',
 'REMOTEIP': '172.130.400.165',
 'PAGENAME': 'NewReports',
 'CREATEDON': Timestamp('2021-10-28 09:51:24.870000')}

In [12]:
# insert all the data
for k, row in df.iterrows():
    collection.insert_one(row.to_dict())

## Perform some simple queries

In [52]:
db.list_collection_names()

['rawdata']

In [20]:
# see how many rows there are
len(list(db.rawdata.find()))

19057

In [48]:
# look at one raw record
list(db.rawdata.find({"ACTIVITYID": 1875083}))

[{'_id': ObjectId('6269ae3069ffd1901615fae5'),
  'ACTIVITYID': 1875083,
  'PGID': 40081,
  'OID': 100,
  'RPID': 0,
  'PATID': 0,
  'USERID': 'USER40081',
  'SESSIONID': 'pgdwm055edgvwqf5kb2rzg55',
  'WEBSERVERIP': 'XXX.XX.XXX.110',
  'REMOTEIP': '172.130.400.165',
  'PAGENAME': 'SearchPatients',
  'CREATEDON': datetime.datetime(2019, 3, 24, 18, 4, 29, 77000)}]

In [34]:
# how many sessions
len(db.rawdata.distinct('SESSIONID'))

1239

In [44]:
# how many pages per user
list(db.rawdata.aggregate([
                     { "$group": { "_id": "$USERID", "pages": { "$count": {} } } },
                     { "$sort": { "pages": -1 } }
                   ]))

[{'_id': 'USER_N', 'pages': 16601},
 {'_id': 'USER40081', 'pages': 1245},
 {'_id': 'USER_1', 'pages': 696},
 {'_id': 'BPKUSER', 'pages': 265},
 {'_id': 'USER_4', 'pages': 133},
 {'_id': 'USER_PRO', 'pages': 88},
 {'_id': 'USER_3', 'pages': 20},
 {'_id': 'USER_2', 'pages': 9}]

## Anonymize user id

In [49]:
db.rawdata.distinct('USERID')

['BPKUSER',
 'USER40081',
 'USER_1',
 'USER_2',
 'USER_3',
 'USER_4',
 'USER_N',
 'USER_PRO']

In [31]:
db.rawdata.update_many(
    {"USERID": 'EN-USER'}, 
    {"$set": {"USERID": "USER_1"}})
db.rawdata.update_many(
    {"USERID": 'TESTRAHUL4'}, 
    {"$set": {"USERID": "USER_3"}})
db.rawdata.update_many(
    {"USERID": 'USERVBS'}, 
    {"$set": {"USERID": "USER_4"}})